In [ ]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import csv 

artist_index_url = "https://bandcamp.com/artist_index" #Set the url for all artists
html = urllib.request.urlopen(artist_index_url) # Open the page
soup = BeautifulSoup(html,'html.parser') #Use beautifulSoup to access html  

# Init variables to be stored in CSV file
ARTIST_ = []
ALBUM_ = []
SONG_ = []
LENGTH_ = []
LOCATION_ = []
related_tags = []


# Get the maximum page value to use as the end point (This is probably a waste of a function)
def get_max_pages(soup):
    n_pages = []
    for page in soup.find_all("a",{"rel":"nofollow"}):
        n_pages.append(int(page.text))
    max_pages = max(n_pages)
    return max_pages

max_pages = get_max_pages(soup)
print(max_pages)

######
###################################################
def get_artist_info(artist_soup):
    
    # Get Artist Name
    artist_name = artist_soup.find("span",{"itemprop":"byArtist"}).text
    artist_name.replace("\\n"," ")
            
    # Get Album Name
    album_name = artist_soup.find("h2",{"class":"trackTitle"}).text
    album_name.replace("\r\n"," ")
        
    # Get Artist Location
    artist_loc = artist_soup.find("span",{"class":"location secondaryText"}).text
    print(artist_loc)
    
    # Get the date
    date = artist_soup.find("meta",{"itemprop":"datePublished"})["content"]
        
    # Get Album Tags
    for tags in artist_soup.find_all("a",{"class":"tag"}): 
        TAGS_.append(tags.text)
            
    # Get Songs
    for song in artist_soup.find_all("span", {"itemprop":"name"}):
        SONG_.append(song.text)       
      
    output = [artist_name,album_name,artist_loc,date,TAGS_,SONG_]    
    return output
################################################################


# Create/Open the CSV File you will be writing too!
with open('Bandcamp_MASTER.csv','w') as csvfile:
    fieldnames = ['Artist','Album','Song','Length','Location','Tags']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames, lineterminator = '\n')
    writer.writeheader()
    
    page_url = "?page=1" # Initialize the 1st page as default
    
    # This will loop for every page available (1-->max_pages)
    for page in range(1, max_pages):
        
        # Get the url for the page number and open it with BeautifulSoup
        page_url = page_url.replace(page_url[len(page_url) - 1], str(page)) # replace the previous page number    
        page_html = artist_index_url + page_url # create the new link for the next page to scan
        page_html = urllib.request.urlopen(page_html) # open the next page
        soup = BeautifulSoup(page_html,'html.parser') # Pull the data from the next page 
        
       # Get the artists on that page and append it to the complete artist list

        artist_class = soup.find('ul', {"class":"item_list"}) # Finds the html section that includes all artist links
        for a_link in artist_class.find_all('a', href = True):
            print(a_link["href"]) # print the artist's url
            artist_url = a_link["href"] # create variable for artist's url
            artist_html = urllib.request.urlopen(artist_url) # open the artist's url
            artist_soup = BeautifulSoup(artist_html,'html.parser') # Pull the data from artist's url     
            
            try:                                    
                album_soup = artist_soup.find('ol', {"data-edit-callback" : "/music_reorder"})
                for l in album_soup.find_all('a',href = True):
                    # checks if it's just a single song or an album
                    if "track" in l["href"]: # It's just a song
                        track_url = artist_url + l["href"]
                        track_html = urllib.request.urlopen(track_url)
                        track_soup = BeautifulSoup(track_html,'html.parser')
                        song_title = track_soup.find("h2",{"class":"trackTitle"}).text
                        song_duraion = track_soup.find("span",{"class":"time_total"}).text #WHY WON't THIS WORK
                        album_title = "N/A"
                        location = track_soup.find("span",{"class":"location secondaryText"}).text
                        date = track_soup.find("meta",{"itemprop":"datePublished"})
                        print(date["content"])
                        for tags in track_soup.find_all("a",{"class":"tag"}): 
                            related_tags.append(tags.text)
                        
                    elif "album" in l["href"]: # It's an album!     
                        print("hey")
                    else:
                        print("OH1")                    
                             
            except: 
                print("OH")